In [1]:
1+1 
#sanity check

In [2]:
spark


In [3]:
import tweepy
from tweepy import OAuthHandler
import json
import numpy as np
import pandas as pd
from kafka import SimpleProducer, KafkaClient
from kafka import KafkaProducer


consumer_key = 'xxx'
consumer_secret = 'xxx'
access_token = 'xxx'
access_secret = 'xxx'

 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)



In [4]:
#get a single tweet
#get tweets from own timeline
for status in tweepy.Cursor(api.home_timeline).items(10):
    print(status.text) 

In [5]:
def process_or_store(tweet):
    print(json.dumps(tweet))

In [6]:
import json
for status in tweepy.Cursor(api.home_timeline).items(10):
    # Process a single status
    process_or_store(status._json) 
    

In [7]:
#trends= api.trends_available()
#print trends
api.trends_available()

In [8]:
query = '#ObamaFarewell'
cursor = tweepy.Cursor(api.search, q=query, lang="en")

In [9]:
import time
for page in cursor.pages(2):
  tweets = []
for item in page:
  TW= tweets.append(item._json)
  print(TW)
  time.sleep(2)

In [10]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler, Stream
#from elasticsearch import Elasticsearch
from datetime import datetime
from textblob import TextBlob


auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)


"""pk = sc.textFile("/FileStore/tables/7f8jfm6e1485266500976/service.cert")
ct = sc.textFile("/FileStore/tables/7f8jfm6e1485266500976/service.key")
ca = sc.textFile("/FileStore/tables/e2n5myz11485269460096/ca.pem")
"""
#es = Elasticsearch([elasticsearch_uri e.g  http://192.168.99.100:49156])

aces = []
thor= {}

class Listener(StreamListener):
  
  def on_data(self, data):
      
    all_data= json.loads(data)
    
    tweet= all_data["text"].encode('utf-8')
    timestamp=all_data["timestamp_ms"].encode('utf-8')
    ## https://docs.python.org/2/library/datetime.html#strftime-and-strptime-behavior
    timestamp_clean = datetime.strptime(all_data["created_at"].replace("+0000 ",""), "%a %b %d %H:%M:%S %Y").isoformat()
    created = all_data["created_at"].encode('utf-8')
    user = all_data["user"]["screen_name"].encode('utf-8')
    followers = all_data["user"]["followers_count"]
    location = all_data["user"]["location"]
    geo= all_data["geo"]
    
    
    
    
    blob= TextBlob(tweet.decode('utf-8'))
    if blob.sentiment.polarity < 0:
      sentiment = "negative"
    elif blob.sentiment.polarity == 0:
      sentiment = "neutral"
    else:
      sentiment = "positive"
    
    kz= [tweet, user]
    thor={'timestamp': timestamp_clean,
         'created_at': created,
         'tweet_text': tweet,
         'screen_name': user,
         'followers': followers,
         'location': location,
         'sentiment_analysis': sentiment
        
         }
    
    #send data to elasticsearch
    """es.create(index= "idx_twp",
                      doc_type= "twitter_twp",
                      body= thor
                     )"""
    
    #hammer =pd.DataFrame.from_dict(thor, orient='index')
    
    
    """from kafka import SimpleProducer, KafkaClient
    from kafka import KafkaConsumer, KafkaProducer
    
    producer = KafkaProducer(bootstrap_servers= ['twitter-fyp.acehanks-7a98.aivencloud.com:27954'],
                              security_protocol="SSL",
                              ssl_keyfile= pk,
                              ssl_certfile= ct,
                              ssl_check_hostname=True,
                              ssl_cafile= ca,
                              api_version=(0,10)
                              )
    #kafka_prod = KafkaProducer(bootstrap_servers='dogsled-01.srvs.cloudkafka.com')
    producer.send('twitter-fyp', b'some_message_bytes' ).get(timeout=60)
    producer.flush()

    for msg in thor:
      #kafka_prod.send('twitter-fyp', b'str(json.dumps(thor))' )
      #kafka_prod.send('vj90-twitter', str(json.dumps(thor)))
      producer.send('twitter-fyp', str(thor) )
      
      print str(json.dumps(thor))"""
      
    #heap= sc.parallelize(thor)
    #heapDF= heap.map(lambda x: (x, )).toDF()
    
    
    
    
    
    #ready = spark.createDataFrame(aces)
    #print all_data
    print thor
    print
    return True #not to stop stream
    
#start stream with tags #syria and #iraq and perform sentiment analaysis  
stream = Stream(auth, Listener())
stream.filter(track=['syria', 'iraq'])
stream.sample()



In [11]:
print thor